In [1]:
import numpy as np
from z3 import *
from ComplexVector import ComplexVector
from complex import Complex, ComplexExpr, I
from scipy.linalg import expm
from IPython.display import clear_output

In [2]:
Equiv = lambda a, b: And(Implies(a, b), Implies(b,a))

In [3]:
from z3 import *

s = Solver()
sys = lambda x: 1/np.sqrt(2) * np.array([x[1] + x[3], -x[0] - x[2], x[1] - x[3], -x[0] + x[2]])

B = Function('b', RealSort(), RealSort(), RealSort(), RealSort(), RealSort())
x = RealVector('x', 4)

# Define barrier and its differential
Bcond = ForAll(x, B(x) == 0.1 - x[0]**2 - x[1]**2)
dBdx = lambda x: np.array([-2 *x[0], -2*x[1], 0, 0])

# Bcond = ForAll(x, B(x) == -(0.9 - x[2]**2 - x[3]**2))
# dBdx = lambda x: np.array([0, 0, 2*x[2], 2*x[3]])
s.add(Bcond)
dBdt = lambda x: np.dot(dBdx(x), sys(x))

# Conditions on x
s.add(x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2 == 1)

# Barrier conditions
def init(x_0):
    return And(x_0[0] ** 2 + x_0[1] ** 2 > 0.9, x_0[0] ** 2 + x_0[1] ** 2 <= 1,
               x_0[2] ** 2 + x_0[3] ** 2 >= 0, x_0[2] ** 2 + x_0[3] ** 2 < 0.1)

def unsafe(x_u):
    return And(x_u[2] ** 2 + x_u[3] ** 2 > 0.9, x_u[2] ** 2 + x_u[3] ** 2 <= 1,
               x_u[0] ** 2 + x_u[1] ** 2 >= 0, x_u[0] ** 2 + x_u[1] ** 2 < 0.1)

s.add(ForAll(x, Implies(init(x), B(x) <= 0)))
s.add(ForAll(x, Implies(unsafe(x), B(x) > 0)))
# s.add(ForAll(x, Implies(B(x) == 0, dBdt(x) <= 0)))
# Find Cex at border
s.add(B(x) == 0, dBdt(x) > 0)

# s.set(timeout=5000)
sat = s.check()
print(sat)
if sat == z3.sat:
    m = s.model()
    print(m)
    p = 0
    for xi in x:
        if is_algebraic_value(m[xi]):
            r = m[xi].approx(20)
        else: r = m[xi]
        if is_rational_value(r):
            r.numerator_as_long()
            r.denominator_as_long()
            v = float(r.numerator_as_long())/float(r.denominator_as_long())
        p += v**2
    print(p)
if sat == z3.unsat: print(s.unsat_core())

sat
[x__3 = 0.9466222847?,
 x__0 = -0.2957066685?,
 x__2 = 1/16,
 x__1 = -459/4096,
 b = [else -> 1/10 + -1*Var(1)**2 + -1*Var(0)**2]]
1.0


In [4]:
# Bloch Sphere setting
from z3 import *

s = Solver()
seed = np.random.randint(0,10000)
s.set(random_seed=seed)
# Change sys
sys = lambda b: _

b = RealVector('b', 3)
bex = RealVector('bex', 3)

# Define barrier and its differential
B = lambda b: 1/5 - b[0] - b[2]
dBdb = lambda b: np.dot(np.array((1,0,1)), b)

dBdt = lambda b: np.dot(dBdb(b), sys(b))

# Conditions on x
s.add(b[0] ** 2 + b[1] ** 2 + b[2] ** 2 == 1)

# Barrier conditions
# Touch surface and z is above/below threshold
def inside(x):
    return And(x[0] ** 2 + x[1] ** 2 + x[2]**2 == 1)

def init(x_0):
    return And(x_0[0] ** 2 + x_0[1] ** 2 + x_0[2]**2 == 1, 4/5 < x_0[2])

def unsafe(x_u):
    return And(x_u[0] ** 2 + x_u[1] ** 2 + x_u[2]**2 == 1, -4/5 > x_u[2])

s.add(ForAll(b, Implies(init(b), B(b) <= 0)))
s.add(ForAll(b, Implies(unsafe(b), B(b) > 0)))
s.add(ForAll(b, dBdt(b) <= 0))
# Find Cex at border
s.add(B(bex) == 0, inside(bex))

# s.set(timeout=5000)
sat = s.check()
print(sat)
if sat == z3.sat:
    m = s.model()
    print(m)
    for xi in x:
        if is_algebraic_value(m[xi]):
            r = m[xi].approx(20)
        else: r = m[xi]
        if is_rational_value(r):
            r.numerator_as_long()
            r.denominator_as_long()
            v = float(r.numerator_as_long())/float(r.denominator_as_long())

if sat == z3.unsat: print(s.unsat_core())

sat
[b__2 = -1/2,
 bex__1 = -0.9893179468?,
 bex__0 = 3/40,
 b__1 = 0,
 bex__2 = 1/8,
 b__0 = -0.8660254037?]


In [5]:
# Given state, barrier is real
# s.add(state[0].len_sqr() + state[1].len_sqr() == 1, B(state[0], state[1]).i != 0)
# Yep!

# Given initials, barrier is real and <= 0
# s.add(state[0].len_sqr() >= 0.9, state[0].len_sqr() + state[1].len_sqr() == 1)
# s.add(Not(And(B(state[0], state[1]).r <= 0, B(state[0], state[1]).i == 0)))
# Yep!

# Given unsafe, barrier is real and > 0
# s.add(state[0].len_sqr() <= 0.1, state[0].len_sqr() + state[1].len_sqr() == 1)
# s.add(Not(And(B(state[0], state[1]).r > 0, B(state[0], state[1]).i == 0)))
# Yep!

# Given a state, dBdt == 0
# s.add(state[0].len_sqr() + state[1].len_sqr() == 1, 
#       Not(And(dBdt(state[0], state[1]).r == 0, dBdt(state[0], state[1]).i == 0))
#      )
# Yep!

In [6]:
import numpy as np
from z3 import *
from complex import Complex, ComplexExpr, I

# Complex setting
# Definition of dynamical system and complex conjugate
f = lambda a,b: np.array([0 - I * (a + b)/Sqrt(2), 0 - I * (a - b)/Sqrt(2)])
fbar = lambda a,b: np.array([I * (a.conj() + b.conj())/Sqrt(2), I * (a.conj() - b.conj())/Sqrt(2)])

# Definition of barrier and its differential
B = lambda a, b: 1.2 - 2 * a * a.conj() - a.conj() * b - a * b.conj()
dBdz = lambda a, b: np.array([0 - a.conj() - b.conj(), b.conj() - a.conj()])
dBdzconj = lambda a, b: np.array([0 - a - b, b - a])
dBdt = lambda a, b: np.dot(dBdz(a, b), f(a, b)) + np.dot(dBdzconj(a, b), fbar(a, b))

# Initialise a Solver and state
s = Solver()
z0 = Complex('z_0')
z1 = Complex('z_1')
state = [z0, z1]

# Add proof constraints
s.add(Or(
    # Barrier is only real on all possible states
    And(state[0].len_sqr() + state[1].len_sqr() == 1, 
        Not(B(state[0], state[1]).i == 0)),
    # Barrier condition for initial region
    And(state[0].len_sqr() >= 0.9, 
        state[0].len_sqr() + state[1].len_sqr() == 1,
        Not(And(B(state[0], state[1]).r <= 0, B(state[0], state[1]).i == 0))),
    # Barrier condition for unsafe region
    And(state[0].len_sqr() < 0.1, 
        state[0].len_sqr() + state[1].len_sqr() == 1,
        Not(And(B(state[0], state[1]).r > 0, B(state[0], state[1]).i == 0))),
    # Barrier condition for convex condition
    And(state[0].len_sqr() + state[1].len_sqr() == 1, 
      Not(And(dBdt(state[0], state[1]).r <= 0, dBdt(state[0], state[1]).i == 0)))
))

# Perform checks
sat = s.check()
print(sat)
if sat == z3.sat:
    m = s.model()
    print(m)
if sat == z3.unsat: print("Barrier meets all conditions")

unsat
Barrier meets all conditions


In [7]:
import numpy as np
from z3 import *
from complex import Complex, ComplexExpr, I

# Sadegh's barrier
u11 = - np.sqrt(2 - np.sqrt(2))
u12 = np.sqrt(2 + np.sqrt(2))
for c in np.arange(-2, 2, .01):
    # Definition of barrier and its differential
    B = lambda a, b: c + u11**2 * a * a.conj() + u11 * u12 * a.conj() * b + u11 * u12 * a * b.conj() + u12**2 * b * b.conj()
    dBdz = lambda a, b: np.array([u11**2 * a.conj() + u11 * u12 * b.conj(), u12**2 * b.conj() + u11 * u12 * a.conj()])
    dBdzconj = lambda a, b: np.array([u11**2 * a + u11 * u12 * b, u12**2 * b + u11 * u12 * a])
    dBdt = lambda a, b: np.dot(dBdz(a, b), f(a, b)) + np.dot(dBdzconj(a, b), fbar(a, b))

    # Initialise a Solver and state
    s = Solver()
    z0 = Complex('z_0')
    z1 = Complex('z_1')
    state = [z0, z1]

    # Add proof constraints
    s.add(Or(
        # Barrier is only real on all possible states
        And(state[0].len_sqr() + state[1].len_sqr() == 1, 
            Not(B(state[0], state[1]).i == 0)),
        # Barrier condition for initial region
        # And(state[0].len_sqr() >= 0.9, 
        #     state[0].len_sqr() + state[1].len_sqr() == 1,
        #     Not(And(
        #         B(state[0], state[1]).r <= 0,
        #         B(state[0], state[1]).i == 0
        #     ))),
        # Barrier condition for unsafe region
        # And(state[0].len_sqr() < 0.1, 
        #     state[0].len_sqr() + state[1].len_sqr() == 1,
        #     Not(And(B(state[0], state[1]).r > 0, B(state[0], state[1]).i == 0))),
        # Barrier condition for convex condition
        And(state[0].len_sqr() + state[1].len_sqr() == 1, 
          Not(And(dBdt(state[0], state[1]).r <= 0, dBdt(state[0], state[1]).i == 0)))
    ))

    # Perform checks
    sat = s.check()
    # if sat == z3.sat:
    #     m = s.model()
    #     print(m)
    clear_output(wait=True)
    if sat == z3.sat: print(c, s.model())
    if sat == z3.unsat:
        print(c, sat)
        print("Barrier meets all conditions")
        time.sleep(5)
        break
print('Done')
if sat == z3.sat: print("No suitable constant")

1.9900000000000038 [z_0.r = 1/8,
 z_0.i = 1/2,
 z_1.i = -0.8569568250?,
 z_1.r = 0]
Done
No suitable constant


In [8]:
from dreal import *

ar = Variable("ar")
ai = Variable("ai")
br = Variable("br")
bi = Variable("bi")
B = Variable("B")

bzr = lambda ar, ai, br, bi: 6/5 - 2 * (ar * ar + ai * ai) - 2 * (ar * br + ai * bi)
f_sat = And(0 <= ar, ar < 1,
            0 <= ai, ai < 1,
            0 <= br, br < 1,
            0 <= bi, bi < 1,
            ar*ar + ai*ai + br*br + bi*bi == 1,
            # ar*ar + ai*ai >= 0.9,
            B == bzr(ar, ai, br, bi), B > 0
           )

result = CheckSatisfiability(f_sat, 0.001)
print(result)

ModuleNotFoundError: No module named 'dreal'

In [ ]:
def bz(alpha, beta):
    return 6/5 - 2 * alpha * np.conj(alpha) - np.conj(alpha) * beta - alpha * np.conj(beta)

bz(np.sqrt(0.9), -np.sqrt(0.1))
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
eiH = lambda t: expm(-1j*t*H)

evo = lambda ts, init: np.array([np.dot(eiH(t), init) for t in ts])

# r = np.arange(np.sqrt(0.9), 1, 0.001)
# ang0 = np.arange(0, 2*np.pi, 0.2)
# ang1 = np.arange(0, 2*np.pi, 0.2)
# phase0 = np.exp(1j*ang0)
# phase1 = np.exp(1j*ang1)
# alphas = np.outer(r, phase0)
# betas = np.outer(np.sqrt(1-r**2), phase1)

# for al, bl in zip(alphas, betas):
#     z0 = np.array([[(a, b) for b in bl] for a in al]).reshape(-1,2)

for i in range(200):
    r = np.random.uniform(np.sqrt(0.9), 1)
    ang0 = np.random.uniform(0, 2*np.pi)
    ang1 = np.random.uniform(0, 2*np.pi)
    z0 = np.array([r * np.exp(1j*ang0), np.sqrt(1-r**2) * np.exp(1j*ang1)])

    t = np.arange(0, 2*np.pi, 0.1)
    z = evo(t, z0).T
    level = bz(z[0][0], z[1][0])
    if not np.all(np.isclose(bz(z[0], z[1]), np.array([level] * len(z[0])))): print("Error: ", z0, level)
    # print(z0, level, )))
print('Done')